**UNZIPPING THE DATA**

In [ ]:
! unzip "../input/dogs-vs-cats-redux-kernels-edition/train.zip"
! unzip "../input/dogs-vs-cats-redux-kernels-edition/test.zip"

**IMPORTING LIBRARIES**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

**DEFINING CONSTANTS**

In [ ]:
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
IMAGE_CHANNELS = 3
BATCH_SIZE = 256

**LOADING DATA**

In [ ]:
filenames = os.listdir("train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == "dog":
        categories.append(1)
    else:
        categories.append(0)

In [ ]:
all_data = pd.DataFrame({
    "filename": filenames,
    "category": categories,
}, dtype = "str")

**SAMPLE IMAGE**

In [ ]:
index = 357
sample_img_filename, sample_img_label = all_data.iloc[index, :]
sample_img_label = int(sample_img_label)
sample_img = plt.imread("train/" + sample_img_filename)
plt.imshow(sample_img)
print("Label: {}({})".format(["Cat", "Dog"][sample_img_label], sample_img_label))

**TRAIN-VALIDATION SPLIT**

In [ ]:
train_data, validation_data = train_test_split(all_data, test_size = 0.05, shuffle = True, random_state = 2)

train_data = train_data.reset_index(drop = True)
validation_data = validation_data.reset_index(drop = True)

train_data.shape, validation_data.shape

In [ ]:
num_train = train_data.shape[0]
num_val = validation_data.shape[0]

**IMPORTING LIBRARIES FOR DEFINING THE MODEL**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization, Activation

**DEFINING IMAGE GENERATORS**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 15,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory = "train/",
    x_col = "filename",
    y_col = "category",
    class_mode = "binary",
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validation_data,
    directory = "train/",
    x_col = "filename",
    y_col = "category",
    class_mode = "binary",
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
)

**EXAMPLE GENERATOR**

In [ ]:
example_generator = train_datagen.flow_from_dataframe(
    train_data.sample(n = 1),
    directory = "train/",
    x_col = "filename",
    y_col = "category",
    target_size = IMAGE_SIZE,
    batch_size = 15,
)

In [ ]:
plt.figure(figsize = (12, 12))
for i in range(15):
    example_data = next(example_generator)
    plt.subplot(5, 3, i + 1)
    image = np.squeeze(example_data[0])
    plt.imshow(image)
    
label = int(example_data[1])
print("Label: {}({})".format(["Cat", "Dog"][label], label))

**DEFINING PRETRAINED VGG19 MODEL**

In [ ]:
pretrained_base = VGG19(
    include_top = False, 
    weights = "imagenet",
    input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
    pooling = None,
)

for layer in pretrained_base.layers[:5]:
    layer.trainable = True
for layer in pretrained_base.layers[5:]:
    layer.trainable = False
pretrained_base.summary()

**DEFINING HEAD LAYERS**

In [ ]:
model = Sequential([
    pretrained_base,
    Flatten(),
    Dropout(0.2),
    Dense(512),
    BatchNormalization(),
    Activation("relu"),
    Dropout(0.2),
    Dense(128),
    BatchNormalization(),
    Activation("relu"),
    Dropout(0.2),
    Dense(32),
    BatchNormalization(),
    Activation("relu"),
    Dense(1, activation = "sigmoid"),
])

model.summary()

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(
    x = train_generator, 
    steps_per_epoch = num_train // BATCH_SIZE,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = num_val // BATCH_SIZE,
)

**PREDICTING ON TEST DATASET**

In [ ]:
filenames = os.listdir("test")
test_data = pd.DataFrame({
    "filename": filenames
}, dtype = "str")

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_data,
    directory = "test",
    x_col = "filename",
    y_col = None,
    target_size = IMAGE_SIZE,
    class_mode = None,
    shuffle = False,
    batch_size = 1,
)

In [ ]:
predictions = model.predict(x = test_generator, batch_size = 1, steps = test_data.shape[0], verbose = 1)
predictions.shape

In [ ]:
predictions = np.squeeze(predictions)

**SUBMISSION**

In [ ]:
ids = np.arange(1, test_data.shape[0] + 1, 1)
ids.shape

In [ ]:
submission = pd.DataFrame({
    "id": ids,
    "label": predictions,
})

In [ ]:
submission.to_csv("submission.csv", index = False)